In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import os

In [2]:
SEED = 2222
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
def process_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
def process_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [5]:
Source = Field(tokenize=process_de, init_token='<sos>', eos_token='<eos>', lower=True)
Target = Field(tokenize=process_en, init_token='<sos>', eos_token='<eos>', lower=True)

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(Source, Target))

In [7]:
len(train_data),len(valid_data),len(test_data)

(29000, 1014, 1000)

In [8]:
Source.build_vocab(train_data, min_freq=2)
Target.build_vocab(train_data, min_freq=2)

In [9]:
BATCH_SIZE = 128

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
device.type

'cuda'

In [12]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))#[sent len, batch size]
        outputs, hidden = self.rnn(embedded)#[sent len, batch size, emb dim]
        #outputs -> [sent len, batch size, hid dim * n directions]
        #hidden -> [n layers * n directions, batch size, hid dim]
        return hidden

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        self.out = nn.Linear(emb_dim + hid_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        #input -> [batch size]
        #hidden -> [n layers * n directions, batch size, hid dim]
        #context -> [n layers * n directions, batch size, hid dim]
        
        #hidden -> [1, batch size, hid dim]
        #context -> [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        #input -> [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        #embedded -> [1, batch size, emb dim]
        
        emb_con = torch.cat((embedded, context), dim=2)
        #emb_con -> [1, batch size, emb dim + hid dim]
        
        output, hidden = self.rnn(emb_con, hidden)
        #output -> [sent len, batch size, hid dim * n directions]
        #hidden -> [n layers * n directions, batch size, hid dim]
        #output -> [1, batch size, hid dim]
        #hidden -> [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1)
        #output -> [batch size, emb dim + hid dim * 2]
        
        prediction = self.out(output)
        
        return prediction,hidden

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        #src->[sent len, batch size]
        #trg->[sent len, batch size]
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        target_voc_size = self.decoder.output_dim
        outputs = torch.zeros(max_len, batch_size, target_voc_size).to(self.device)
        context = self.encoder(src)
        hidden = context
        input = trg[0,:]
        for t in range(1, max_len):
            output, hidden = self.decoder(input, hidden, context)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs

In [17]:
INPUT_DIM = len(Source.vocab)
OUTPUT_DIM = len(Target.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [18]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5)
  )
)

In [19]:
optimizer = optim.Adam(model.parameters())

In [20]:
pad_idx = Target.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [21]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        source = batch.src
        target = batch.trg#[sent len, batch size]
        optimizer.zero_grad()
        output = model(source, target)#[sent len, batch size, output dim]
        loss = criterion(output[1:].view(-1, output.shape[2]), target[1:].view(-1))
        #trg->[(sent len - 1) * batch size]
        #output->[(sent len - 1) * batch size, output dim]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [22]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            source = batch.src
            target = batch.trg
            output = model(source, target, 0)
            loss = criterion(output[1:].view(-1, output.shape[2]), target[1:].view(-1))
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [23]:
DIR = 'models'
MODEL_DIR = os.path.join(DIR, 'seq2seq_model.pt')

N_EPOCHS = 10
CLIP = 10

best_loss = float('inf')

if not os.path.isdir(f'{DIR}'):
    os.makedirs(f'{DIR}')
    
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(model.state_dict(), MODEL_DIR)
    print(f'| Epoch: {epoch+1:03} | Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | Val. Loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f} |')

| Epoch: 001 | Train Loss: 4.641 | Train PPL: 103.700 | Val. Loss: 4.439 | Val. PPL:  84.687 |
| Epoch: 002 | Train Loss: 3.666 | Train PPL:  39.111 | Val. Loss: 3.914 | Val. PPL:  50.118 |
| Epoch: 003 | Train Loss: 3.204 | Train PPL:  24.630 | Val. Loss: 3.718 | Val. PPL:  41.171 |
| Epoch: 004 | Train Loss: 2.877 | Train PPL:  17.765 | Val. Loss: 3.605 | Val. PPL:  36.765 |
| Epoch: 005 | Train Loss: 2.672 | Train PPL:  14.471 | Val. Loss: 3.554 | Val. PPL:  34.937 |
| Epoch: 006 | Train Loss: 2.459 | Train PPL:  11.693 | Val. Loss: 3.614 | Val. PPL:  37.126 |
| Epoch: 007 | Train Loss: 2.296 | Train PPL:   9.937 | Val. Loss: 3.581 | Val. PPL:  35.921 |
| Epoch: 008 | Train Loss: 2.195 | Train PPL:   8.984 | Val. Loss: 3.649 | Val. PPL:  38.444 |
| Epoch: 009 | Train Loss: 2.091 | Train PPL:   8.089 | Val. Loss: 3.642 | Val. PPL:  38.174 |
| Epoch: 010 | Train Loss: 2.000 | Train PPL:   7.387 | Val. Loss: 3.679 | Val. PPL:  39.615 |


In [24]:
model.load_state_dict(torch.load(MODEL_DIR))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.525 | Test PPL:  33.950 |
